In [1]:
# This is the second python code file used for Ethical Princiles comparison, making use of the AIML techniques. 


# 1. Using transformers to calculate cosine similarity between ethical principles

In [ ]:
cd /content/drive/MyDrive/MLFinanceEthics/results

In [ ]:
mkdir results

In [ ]:
!ls

In [5]:
import pandas as pd
from itertools import product

## Read dataset

In [6]:
# Retrieve long and short definitions of ethical principles in finance and machine learning
fin_dataset = pd.read_excel("dataset_ethics.xlsx",sheet_name='Finance')
ml_dataset = pd.read_excel("dataset_ethics.xlsx",sheet_name='ML')

In [7]:
# Print the ethical principles of machine learning
ml_dataset.head()

,Principle,Long definition,Short definition
0,"Inclusive growth, sustainable development and ...",This principle states that AI should be develo...,Trustworthy AI should contribute to overall gr...
1,Human-centered values and fairness,"Based on this principle, AI should be develope...",AI systems should be designed in a way that re...
2,Transparency and explainability,Transparency defined in this principle has two...,Transparent and responsible disclosure around ...
3,"Robustness, security and safety",This principle states that AI systems must be ...,"AI systems must function in a robust, secure a..."
4,Accountability,"\nAccording to this principle, organisations a...","Organisations and individuals developing, depl..."


In [8]:
# Print the ethical principles of finance
fin_dataset.head()

,Principle,Long definition,Short definition
0,Integrity,Acting with integrity is one of the main princ...,"Moral self-governance, autonomy, trustworthine..."
1,Objectivity,Objectivity is ground on the subordination of ...,Protecting and advancing the interests of clie...
2,Competence,Professionals are obligated by to maintain the...,Rendering competent financial services to clie...
3,Fairness,The principle of fairness is an integral part ...,"Treating customers equitably, consistently app..."
4,Confidentiality,Confidentiality is the obligation to hold clie...,"Handling client relationships with confidence,..."


In [9]:
ds_principles = pd.DataFrame(list(product(fin_dataset['Principle'], ml_dataset['Principle'])),columns=["Finance","ML"])
ds_short_defs = pd.DataFrame(list(product(fin_dataset['Short definition'], ml_dataset['Short definition'])),columns=["Finance","ML"])
ds_long_defs = pd.DataFrame(list(product(fin_dataset['Long definition'], ml_dataset['Long definition'])),columns=["Finance","ML"])

In [ ]:
!pip install transformers
!pip install sentence-transformers

In [11]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

## Define transformer models

In [13]:
# Define transformer models that will be used in the experiment
models=['sentence-transformers/LaBSE','sentence-transformers/allenai-specter',
        'sentence-transformers/average_word_embeddings_glove.6B.300d','sentence-transformers/average_word_embeddings_glove.840B.300d',
        'sentence-transformers/average_word_embeddings_komninos','sentence-transformers/average_word_embeddings_levy_dependency',
        'sentence-transformers/bert-base-nli-cls-token','sentence-transformers/bert-base-nli-max-tokens',
        'sentence-transformers/bert-base-nli-mean-tokens','sentence-transformers/bert-base-nli-stsb-mean-tokens',
        'sentence-transformers/bert-base-wikipedia-sections-mean-tokens','sentence-transformers/bert-large-nli-cls-token',
        'sentence-transformers/bert-large-nli-max-tokens','sentence-transformers/bert-large-nli-mean-tokens',
        'sentence-transformers/bert-large-nli-stsb-mean-tokens','sentence-transformers/bert-large-nli-mean-tokens',
        'sentence-transformers/distilbert-base-nli-max-tokens',
        'sentence-transformers/distilbert-base-nli-mean-tokens','sentence-transformers/distilbert-base-nli-stsb-mean-tokens',
        'sentence-transformers/distilbert-base-nli-stsb-quora-ranking','sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking',
        'sentence-transformers/distilroberta-base-msmarco-v1','sentence-transformers/distilroberta-base-msmarco-v2',
        'sentence-transformers/distilroberta-base-paraphrase-v1','sentence-transformers/distiluse-base-multilingual-cased-v1',
        'sentence-transformers/distiluse-base-multilingual-cased-v2','sentence-transformers/distiluse-base-multilingual-cased',
        'sentence-transformers/facebook-dpr-ctx_encoder-multiset-base','sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base',
        'sentence-transformers/facebook-dpr-question_encoder-multiset-base','sentence-transformers/facebook-dpr-question_encoder-single-nq-base',
        'sentence-transformers/msmarco-MiniLM-L-12-v3','sentence-transformers/msmarco-MiniLM-L-6-v3',
        'sentence-transformers/msmarco-distilbert-base-dot-prod-v3','sentence-transformers/msmarco-distilbert-base-tas-b',
        'sentence-transformers/msmarco-distilbert-base-v2','sentence-transformers/msmarco-distilbert-base-v3',
        'sentence-transformers/msmarco-distilbert-base-v4','sentence-transformers/msmarco-distilbert-multilingual-en-de-v2-tmp-lng-aligned',
        'sentence-transformers/msmarco-distilbert-multilingual-en-de-v2-tmp-trained-scratch','sentence-transformers/msmarco-distilroberta-base-v2',
        'sentence-transformers/msmarco-roberta-base-ance-firstp','sentence-transformers/msmarco-roberta-base-v2',
        'sentence-transformers/msmarco-roberta-base-v3','sentence-transformers/nli-bert-base-cls-pooling',
        'sentence-transformers/nli-bert-base-max-pooling','sentence-transformers/nli-bert-base',
        'sentence-transformers/nli-bert-large-cls-pooling','sentence-transformers/nli-bert-large-max-pooling',
        'sentence-transformers/nli-bert-large','sentence-transformers/nli-distilbert-base-max-pooling',
        'sentence-transformers/nli-distilbert-base','sentence-transformers/nli-distilroberta-base-v2',
        'sentence-transformers/nli-mpnet-base-v2','sentence-transformers/nli-roberta-base-v2',
        'sentence-transformers/nli-roberta-base','sentence-transformers/nli-roberta-large',
        'sentence-transformers/nq-distilbert-base-v1','sentence-transformers/paraphrase-MiniLM-L12-v2',
        'sentence-transformers/paraphrase-MiniLM-L3-v2','sentence-transformers/paraphrase-MiniLM-L6-v2',
        'sentence-transformers/paraphrase-TinyBERT-L6-v2','sentence-transformers/paraphrase-albert-base-v2',
        'sentence-transformers/paraphrase-albert-small-v2','sentence-transformers/paraphrase-distilroberta-base-v1',
        'sentence-transformers/paraphrase-distilroberta-base-v2','sentence-transformers/paraphrase-mpnet-base-v2',
        'sentence-transformers/stsb-bert-base','sentence-transformers/stsb-bert-large',
        'sentence-transformers/stsb-distilbert-base','sentence-transformers/stsb-distilroberta-base-v2',
        'sentence-transformers/stsb-mpnet-base-v2','sentence-transformers/stsb-roberta-base-v2',
        'sentence-transformers/stsb-roberta-base','sentence-transformers/stsb-roberta-base',
        'sentence-transformers/stsb-roberta-large','sentence-transformers/stsb-xlm-r-multilingual',
        'sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens','sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens',
        'sentence-transformers/xlm-r-bert-base-nli-mean-tokens','sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens',
        'sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1',
        'all-mpnet-base-v2']

## Calculate cosine similarity across all transformers

In [14]:
# Calculate cosine similarity between ethical principles across all transformers
def calc_sim(row, model,label):
  sentence1 = row['Finance']
  sentence2 = row['ML']
  embedding1 = model.encode(sentence1, convert_to_tensor=True)
  embedding2 = model.encode(sentence2, convert_to_tensor=True)
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  row[label]=cosine_scores.item()
  return row;

In [ ]:
# Perform the experiment
for model_name in models:
  print("MODEL NAME:"+model_name)
  model = SentenceTransformer(model_name)
  ds_principles = ds_principles.apply(lambda row: calc_sim(row,model,model_name),axis=1)
  ds_short_defs = ds_short_defs.apply(lambda row: calc_sim(row,model,model_name),axis=1)
  ds_long_defs = ds_long_defs.apply(lambda row: calc_sim(row,model,model_name),axis=1)
  ds_principles.to_pickle("./results/ds_principles.pickle")
  ds_short_defs.to_pickle("./results/ds_short_defs.pickle")
  ds_long_defs.to_pickle("./results/ds_long_defs.pickle")

In [16]:
ds_principles = ds_principles.apply(lambda row: calc_sim(row,model,'stsb-roberta-large'),axis=1)
ds_short_defs = ds_short_defs.apply(lambda row: calc_sim(row,model,'stsb-roberta-large'),axis=1)
ds_long_defs = ds_long_defs.apply(lambda row: calc_sim(row,model,'stsb-roberta-large'),axis=1)

In [17]:
ds_short_defs.head()

,Finance,ML,sentence-transformers/LaBSE,sentence-transformers/allenai-specter,sentence-transformers/average_word_embeddings_glove.6B.300d,sentence-transformers/average_word_embeddings_glove.840B.300d,sentence-transformers/average_word_embeddings_komninos,sentence-transformers/average_word_embeddings_levy_dependency,sentence-transformers/bert-base-nli-cls-token,sentence-transformers/bert-base-nli-max-tokens,sentence-transformers/bert-base-nli-mean-tokens,sentence-transformers/bert-base-nli-stsb-mean-tokens,sentence-transformers/bert-base-wikipedia-sections-mean-tokens,sentence-transformers/bert-large-nli-cls-token,sentence-transformers/bert-large-nli-max-tokens,sentence-transformers/bert-large-nli-mean-tokens,sentence-transformers/bert-large-nli-stsb-mean-tokens,sentence-transformers/distilbert-base-nli-max-tokens,sentence-transformers/distilbert-base-nli-mean-tokens,sentence-transformers/distilbert-base-nli-stsb-mean-tokens,sentence-transformers/distilbert-base-nli-stsb-quora-ranking,sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking,sentence-transformers/distilroberta-base-msmarco-v1,sentence-transformers/distilroberta-base-msmarco-v2,sentence-transformers/distilroberta-base-paraphrase-v1,sentence-transformers/distiluse-base-multilingual-cased-v1,sentence-transformers/distiluse-base-multilingual-cased-v2,sentence-transformers/distiluse-base-multilingual-cased,sentence-transformers/facebook-dpr-ctx_encoder-multiset-base,sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base,sentence-transformers/facebook-dpr-question_encoder-multiset-base,sentence-transformers/facebook-dpr-question_encoder-single-nq-base,sentence-transformers/msmarco-MiniLM-L-12-v3,sentence-transformers/msmarco-MiniLM-L-6-v3,sentence-transformers/msmarco-distilbert-base-dot-prod-v3,sentence-transformers/msmarco-distilbert-base-tas-b,sentence-transformers/msmarco-distilbert-base-v2,sentence-transformers/msmarco-distilbert-base-v3,sentence-transformers/msmarco-distilbert-base-v4,sentence-transformers/msmarco-distilbert-multilingual-en-de-v2-tmp-lng-aligned,...,sentence-transformers/msmarco-roberta-base-v3,sentence-transformers/nli-bert-base-cls-pooling,sentence-transformers/nli-bert-base-max-pooling,sentence-transformers/nli-bert-base,sentence-transformers/nli-bert-large-cls-pooling,sentence-transformers/nli-bert-large-max-pooling,sentence-transformers/nli-bert-large,sentence-transformers/nli-distilbert-base-max-pooling,sentence-transformers/nli-distilbert-base,sentence-transformers/nli-distilroberta-base-v2,sentence-transformers/nli-mpnet-base-v2,sentence-transformers/nli-roberta-base-v2,sentence-transformers/nli-roberta-base,sentence-transformers/nli-roberta-large,sentence-transformers/nq-distilbert-base-v1,sentence-transformers/paraphrase-MiniLM-L12-v2,sentence-transformers/paraphrase-MiniLM-L3-v2,sentence-transformers/paraphrase-MiniLM-L6-v2,sentence-transformers/paraphrase-TinyBERT-L6-v2,sentence-transformers/paraphrase-albert-base-v2,sentence-transformers/paraphrase-albert-small-v2,sentence-transformers/paraphrase-distilroberta-base-v1,sentence-transformers/paraphrase-distilroberta-base-v2,sentence-transformers/paraphrase-mpnet-base-v2,sentence-transformers/stsb-bert-base,sentence-transformers/stsb-bert-large,sentence-transformers/stsb-distilbert-base,sentence-transformers/stsb-distilroberta-base-v2,sentence-transformers/stsb-mpnet-base-v2,sentence-transformers/stsb-roberta-base-v2,sentence-transformers/stsb-roberta-base,sentence-transformers/stsb-roberta-large,sentence-transformers/stsb-xlm-r-multilingual,sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens,sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens,sentence-transformers/xlm-r-bert-base-nli-mean-tokens,sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens,sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1,all-mpnet-base-v2,stsb-roberta-large
0,"Moral self-governance, autonomy, trustworthine...",Trustworthy AI should contribute to o

## Save Results

In [18]:
import pandas as pd
ds_principles = pd.read_pickle("/content/results/ds_principles.pickle")
ds_long_defs = pd.read_pickle("/content/results/ds_long_defs.pickle")
ds_short_defs = pd.read_pickle("/content/results/ds_short_defs.pickle")

In [19]:
# Save the results
ds_principles.to_excel("ds_principles.xlsx")
ds_long_defs.to_excel("ds_long_defs.xlsx")
ds_short_defs.to_excel("ds_short_defs.xlsx")

# 2. Comparison between ethical principles of finance and machine learning

## Long and short definitions of ethical principles

In [20]:
long_defs = pd.read_excel('/content/ds_long_defs.xlsx')

In [21]:
long_defs.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

In [22]:
long_defs

,Finance,ML,sentence-transformers/LaBSE,sentence-transformers/allenai-specter,sentence-transformers/average_word_embeddings_glove.6B.300d,sentence-transformers/average_word_embeddings_glove.840B.300d,sentence-transformers/average_word_embeddings_komninos,sentence-transformers/average_word_embeddings_levy_dependency,sentence-transformers/bert-base-nli-cls-token,sentence-transformers/bert-base-nli-max-tokens,sentence-transformers/bert-base-nli-mean-tokens,sentence-transformers/bert-base-nli-stsb-mean-tokens,sentence-transformers/bert-base-wikipedia-sections-mean-tokens,sentence-transformers/bert-large-nli-cls-token,sentence-transformers/bert-large-nli-max-tokens,sentence-transformers/bert-large-nli-mean-tokens,sentence-transformers/bert-large-nli-stsb-mean-tokens,sentence-transformers/distilbert-base-nli-max-tokens,sentence-transformers/distilbert-base-nli-mean-tokens,sentence-transformers/distilbert-base-nli-stsb-mean-tokens,sentence-transformers/distilbert-base-nli-stsb-quora-ranking,sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking,sentence-transformers/distilroberta-base-msmarco-v1,sentence-transformers/distilroberta-base-msmarco-v2,sentence-transformers/distilroberta-base-paraphrase-v1,sentence-transformers/distiluse-base-multilingual-cased-v1,sentence-transformers/distiluse-base-multilingual-cased-v2,sentence-transformers/distiluse-base-multilingual-cased,sentence-transformers/facebook-dpr-ctx_encoder-multiset-base,sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base,sentence-transformers/facebook-dpr-question_encoder-multiset-base,sentence-transformers/facebook-dpr-question_encoder-single-nq-base,sentence-transformers/msmarco-MiniLM-L-12-v3,sentence-transformers/msmarco-MiniLM-L-6-v3,sentence-transformers/msmarco-distilbert-base-dot-prod-v3,sentence-transformers/msmarco-distilbert-base-tas-b,sentence-transformers/msmarco-distilbert-base-v2,sentence-transformers/msmarco-distilbert-base-v3,sentence-transformers/msmarco-distilbert-base-v4,sentence-transformers/msmarco-distilbert-multilingual-en-de-v2-tmp-lng-aligned,...,sentence-transformers/msmarco-roberta-base-v2,sentence-transformers/msmarco-roberta-base-v3,sentence-transformers/nli-bert-base-cls-pooling,sentence-transformers/nli-bert-base-max-pooling,sentence-transformers/nli-bert-base,sentence-transformers/nli-bert-large-cls-pooling,sentence-transformers/nli-bert-large-max-pooling,sentence-transformers/nli-bert-large,sentence-transformers/nli-distilbert-base-max-pooling,sentence-transformers/nli-distilbert-base,sentence-transformers/nli-distilroberta-base-v2,sentence-transformers/nli-mpnet-base-v2,sentence-transformers/nli-roberta-base-v2,sentence-transformers/nli-roberta-base,sentence-transformers/nli-roberta-large,sentence-transformers/nq-distilbert-base-v1,sentence-transformers/paraphrase-MiniLM-L12-v2,sentence-transformers/paraphrase-MiniLM-L3-v2,sentence-transformers/paraphrase-MiniLM-L6-v2,sentence-transformers/paraphrase-TinyBERT-L6-v2,sentence-transformers/paraphrase-albert-base-v2,sentence-transformers/paraphrase-albert-small-v2,sentence-transformers/paraphrase-distilroberta-base-v1,sentence-transformers/paraphrase-distilroberta-base-v2,sentence-transformers/paraphrase-mpnet-base-v2,sentence-transformers/stsb-bert-base,sentence-transformers/stsb-bert-large,sentence-transformers/stsb-distilbert-base,sentence-transformers/stsb-distilroberta-base-v2,sentence-transformers/stsb-mpnet-base-v2,sentence-transformers/stsb-roberta-base-v2,sentence-transformers/stsb-roberta-base,sentence-transformers/stsb-roberta-large,sentence-transformers/stsb-xlm-r-multilingual,sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens,sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens,sentence-transformers/xlm-r-bert-base-nli-mean-tokens,sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens,sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1,all-mpnet-base-v2
0,Acting with integrity is one of the main princ...,This princip

In [23]:
short_defs = pd.read_excel('/content/ds_short_defs.xlsx')

In [24]:
short_defs.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

In [25]:
short_defs

,Finance,ML,sentence-transformers/LaBSE,sentence-transformers/allenai-specter,sentence-transformers/average_word_embeddings_glove.6B.300d,sentence-transformers/average_word_embeddings_glove.840B.300d,sentence-transformers/average_word_embeddings_komninos,sentence-transformers/average_word_embeddings_levy_dependency,sentence-transformers/bert-base-nli-cls-token,sentence-transformers/bert-base-nli-max-tokens,sentence-transformers/bert-base-nli-mean-tokens,sentence-transformers/bert-base-nli-stsb-mean-tokens,sentence-transformers/bert-base-wikipedia-sections-mean-tokens,sentence-transformers/bert-large-nli-cls-token,sentence-transformers/bert-large-nli-max-tokens,sentence-transformers/bert-large-nli-mean-tokens,sentence-transformers/bert-large-nli-stsb-mean-tokens,sentence-transformers/distilbert-base-nli-max-tokens,sentence-transformers/distilbert-base-nli-mean-tokens,sentence-transformers/distilbert-base-nli-stsb-mean-tokens,sentence-transformers/distilbert-base-nli-stsb-quora-ranking,sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking,sentence-transformers/distilroberta-base-msmarco-v1,sentence-transformers/distilroberta-base-msmarco-v2,sentence-transformers/distilroberta-base-paraphrase-v1,sentence-transformers/distiluse-base-multilingual-cased-v1,sentence-transformers/distiluse-base-multilingual-cased-v2,sentence-transformers/distiluse-base-multilingual-cased,sentence-transformers/facebook-dpr-ctx_encoder-multiset-base,sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base,sentence-transformers/facebook-dpr-question_encoder-multiset-base,sentence-transformers/facebook-dpr-question_encoder-single-nq-base,sentence-transformers/msmarco-MiniLM-L-12-v3,sentence-transformers/msmarco-MiniLM-L-6-v3,sentence-transformers/msmarco-distilbert-base-dot-prod-v3,sentence-transformers/msmarco-distilbert-base-tas-b,sentence-transformers/msmarco-distilbert-base-v2,sentence-transformers/msmarco-distilbert-base-v3,sentence-transformers/msmarco-distilbert-base-v4,sentence-transformers/msmarco-distilbert-multilingual-en-de-v2-tmp-lng-aligned,...,sentence-transformers/msmarco-roberta-base-v2,sentence-transformers/msmarco-roberta-base-v3,sentence-transformers/nli-bert-base-cls-pooling,sentence-transformers/nli-bert-base-max-pooling,sentence-transformers/nli-bert-base,sentence-transformers/nli-bert-large-cls-pooling,sentence-transformers/nli-bert-large-max-pooling,sentence-transformers/nli-bert-large,sentence-transformers/nli-distilbert-base-max-pooling,sentence-transformers/nli-distilbert-base,sentence-transformers/nli-distilroberta-base-v2,sentence-transformers/nli-mpnet-base-v2,sentence-transformers/nli-roberta-base-v2,sentence-transformers/nli-roberta-base,sentence-transformers/nli-roberta-large,sentence-transformers/nq-distilbert-base-v1,sentence-transformers/paraphrase-MiniLM-L12-v2,sentence-transformers/paraphrase-MiniLM-L3-v2,sentence-transformers/paraphrase-MiniLM-L6-v2,sentence-transformers/paraphrase-TinyBERT-L6-v2,sentence-transformers/paraphrase-albert-base-v2,sentence-transformers/paraphrase-albert-small-v2,sentence-transformers/paraphrase-distilroberta-base-v1,sentence-transformers/paraphrase-distilroberta-base-v2,sentence-transformers/paraphrase-mpnet-base-v2,sentence-transformers/stsb-bert-base,sentence-transformers/stsb-bert-large,sentence-transformers/stsb-distilbert-base,sentence-transformers/stsb-distilroberta-base-v2,sentence-transformers/stsb-mpnet-base-v2,sentence-transformers/stsb-roberta-base-v2,sentence-transformers/stsb-roberta-base,sentence-transformers/stsb-roberta-large,sentence-transformers/stsb-xlm-r-multilingual,sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens,sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens,sentence-transformers/xlm-r-bert-base-nli-mean-tokens,sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens,sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1,all-mpnet-base-v2
0,"Moral self-governance, autonomy, trustworthine...",Trustworth

## Helper functions

In [26]:
# Calculate percentiles
def get_percentile(dataset, column, percentile):
  return np.percentile(dataset[column].values, percentile)

In [27]:
# Annotate relationship based on percentiles
def annotate_strength_relationship(dataset):
  new_dataset = pd.DataFrame()

  for column in dataset.columns:
    l = []
    percentile33 = get_percentile(dataset, column, 33.333)
    percentile66 = get_percentile(dataset, column, 66.666)

    for item in dataset[column].values:
      if item <= percentile33:
        l.append(0)
      elif item <= percentile66:
        l.append(1)
      else:
        l.append(2)

    new_dataset[column] = np.asarray(l)

  return new_dataset

## Mappings between the ethical principles based on long and short definitions

In [28]:
links_long = annotate_strength_relationship(long_defs.iloc[:,2:])

In [29]:
links_long

,sentence-transformers/LaBSE,sentence-transformers/allenai-specter,sentence-transformers/average_word_embeddings_glove.6B.300d,sentence-transformers/average_word_embeddings_glove.840B.300d,sentence-transformers/average_word_embeddings_komninos,sentence-transformers/average_word_embeddings_levy_dependency,sentence-transformers/bert-base-nli-cls-token,sentence-transformers/bert-base-nli-max-tokens,sentence-transformers/bert-base-nli-mean-tokens,sentence-transformers/bert-base-nli-stsb-mean-tokens,sentence-transformers/bert-base-wikipedia-sections-mean-tokens,sentence-transformers/bert-large-nli-cls-token,sentence-transformers/bert-large-nli-max-tokens,sentence-transformers/bert-large-nli-mean-tokens,sentence-transformers/bert-large-nli-stsb-mean-tokens,sentence-transformers/distilbert-base-nli-max-tokens,sentence-transformers/distilbert-base-nli-mean-tokens,sentence-transformers/distilbert-base-nli-stsb-mean-tokens,sentence-transformers/distilbert-base-nli-stsb-quora-ranking,sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking,sentence-transformers/distilroberta-base-msmarco-v1,sentence-transformers/distilroberta-base-msmarco-v2,sentence-transformers/distilroberta-base-paraphrase-v1,sentence-transformers/distiluse-base-multilingual-cased-v1,sentence-transformers/distiluse-base-multilingual-cased-v2,sentence-transformers/distiluse-base-multilingual-cased,sentence-transformers/facebook-dpr-ctx_encoder-multiset-base,sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base,sentence-transformers/facebook-dpr-question_encoder-multiset-base,sentence-transformers/facebook-dpr-question_encoder-single-nq-base,sentence-transformers/msmarco-MiniLM-L-12-v3,sentence-transformers/msmarco-MiniLM-L-6-v3,sentence-transformers/msmarco-distilbert-base-dot-prod-v3,sentence-transformers/msmarco-distilbert-base-tas-b,sentence-transformers/msmarco-distilbert-base-v2,sentence-transformers/msmarco-distilbert-base-v3,sentence-transformers/msmarco-distilbert-base-v4,sentence-transformers/msmarco-distilbert-multilingual-en-de-v2-tmp-lng-aligned,sentence-transformers/msmarco-distilbert-multilingual-en-de-v2-tmp-trained-scratch,sentence-transformers/msmarco-distilroberta-base-v2,...,sentence-transformers/msmarco-roberta-base-v2,sentence-transformers/msmarco-roberta-base-v3,sentence-transformers/nli-bert-base-cls-pooling,sentence-transformers/nli-bert-base-max-pooling,sentence-transformers/nli-bert-base,sentence-transformers/nli-bert-large-cls-pooling,sentence-transformers/nli-bert-large-max-pooling,sentence-transformers/nli-bert-large,sentence-transformers/nli-distilbert-base-max-pooling,sentence-transformers/nli-distilbert-base,sentence-transformers/nli-distilroberta-base-v2,sentence-transformers/nli-mpnet-base-v2,sentence-transformers/nli-roberta-base-v2,sentence-transformers/nli-roberta-base,sentence-transformers/nli-roberta-large,sentence-transformers/nq-distilbert-base-v1,sentence-transformers/paraphrase-MiniLM-L12-v2,sentence-transformers/paraphrase-MiniLM-L3-v2,sentence-transformers/paraphrase-MiniLM-L6-v2,sentence-transformers/paraphrase-TinyBERT-L6-v2,sentence-transformers/paraphrase-albert-base-v2,sentence-transformers/paraphrase-albert-small-v2,sentence-transformers/paraphrase-distilroberta-base-v1,sentence-transformers/paraphrase-distilroberta-base-v2,sentence-transformers/paraphrase-mpnet-base-v2,sentence-transformers/stsb-bert-base,sentence-transformers/stsb-bert-large,sentence-transformers/stsb-distilbert-base,sentence-transformers/stsb-distilroberta-base-v2,sentence-transformers/stsb-mpnet-base-v2,sentence-transformers/stsb-roberta-base-v2,sentence-transformers/stsb-roberta-base,sentence-transformers/stsb-roberta-large,sentence-transformers/stsb-xlm-r-multilingual,sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens,sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens,sentence-transformers/xlm-r-bert-base-nli-mean-tokens,sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens,sentence-transforme

In [30]:
links_short = annotate_strength_relationship(short_defs.iloc[:,2:])

In [31]:
links_short

,sentence-transformers/LaBSE,sentence-transformers/allenai-specter,sentence-transformers/average_word_embeddings_glove.6B.300d,sentence-transformers/average_word_embeddings_glove.840B.300d,sentence-transformers/average_word_embeddings_komninos,sentence-transformers/average_word_embeddings_levy_dependency,sentence-transformers/bert-base-nli-cls-token,sentence-transformers/bert-base-nli-max-tokens,sentence-transformers/bert-base-nli-mean-tokens,sentence-transformers/bert-base-nli-stsb-mean-tokens,sentence-transformers/bert-base-wikipedia-sections-mean-tokens,sentence-transformers/bert-large-nli-cls-token,sentence-transformers/bert-large-nli-max-tokens,sentence-transformers/bert-large-nli-mean-tokens,sentence-transformers/bert-large-nli-stsb-mean-tokens,sentence-transformers/distilbert-base-nli-max-tokens,sentence-transformers/distilbert-base-nli-mean-tokens,sentence-transformers/distilbert-base-nli-stsb-mean-tokens,sentence-transformers/distilbert-base-nli-stsb-quora-ranking,sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking,sentence-transformers/distilroberta-base-msmarco-v1,sentence-transformers/distilroberta-base-msmarco-v2,sentence-transformers/distilroberta-base-paraphrase-v1,sentence-transformers/distiluse-base-multilingual-cased-v1,sentence-transformers/distiluse-base-multilingual-cased-v2,sentence-transformers/distiluse-base-multilingual-cased,sentence-transformers/facebook-dpr-ctx_encoder-multiset-base,sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base,sentence-transformers/facebook-dpr-question_encoder-multiset-base,sentence-transformers/facebook-dpr-question_encoder-single-nq-base,sentence-transformers/msmarco-MiniLM-L-12-v3,sentence-transformers/msmarco-MiniLM-L-6-v3,sentence-transformers/msmarco-distilbert-base-dot-prod-v3,sentence-transformers/msmarco-distilbert-base-tas-b,sentence-transformers/msmarco-distilbert-base-v2,sentence-transformers/msmarco-distilbert-base-v3,sentence-transformers/msmarco-distilbert-base-v4,sentence-transformers/msmarco-distilbert-multilingual-en-de-v2-tmp-lng-aligned,sentence-transformers/msmarco-distilbert-multilingual-en-de-v2-tmp-trained-scratch,sentence-transformers/msmarco-distilroberta-base-v2,...,sentence-transformers/msmarco-roberta-base-v2,sentence-transformers/msmarco-roberta-base-v3,sentence-transformers/nli-bert-base-cls-pooling,sentence-transformers/nli-bert-base-max-pooling,sentence-transformers/nli-bert-base,sentence-transformers/nli-bert-large-cls-pooling,sentence-transformers/nli-bert-large-max-pooling,sentence-transformers/nli-bert-large,sentence-transformers/nli-distilbert-base-max-pooling,sentence-transformers/nli-distilbert-base,sentence-transformers/nli-distilroberta-base-v2,sentence-transformers/nli-mpnet-base-v2,sentence-transformers/nli-roberta-base-v2,sentence-transformers/nli-roberta-base,sentence-transformers/nli-roberta-large,sentence-transformers/nq-distilbert-base-v1,sentence-transformers/paraphrase-MiniLM-L12-v2,sentence-transformers/paraphrase-MiniLM-L3-v2,sentence-transformers/paraphrase-MiniLM-L6-v2,sentence-transformers/paraphrase-TinyBERT-L6-v2,sentence-transformers/paraphrase-albert-base-v2,sentence-transformers/paraphrase-albert-small-v2,sentence-transformers/paraphrase-distilroberta-base-v1,sentence-transformers/paraphrase-distilroberta-base-v2,sentence-transformers/paraphrase-mpnet-base-v2,sentence-transformers/stsb-bert-base,sentence-transformers/stsb-bert-large,sentence-transformers/stsb-distilbert-base,sentence-transformers/stsb-distilroberta-base-v2,sentence-transformers/stsb-mpnet-base-v2,sentence-transformers/stsb-roberta-base-v2,sentence-transformers/stsb-roberta-base,sentence-transformers/stsb-roberta-large,sentence-transformers/stsb-xlm-r-multilingual,sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens,sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens,sentence-transformers/xlm-r-bert-base-nli-mean-tokens,sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens,sentence-transforme

## Determine overlaps across all transformers

In [32]:
# Calculate the number of overlaps between long and short mappings for each transformer
def calculate_overlaps(long_links, short_links):
  overlaps = pd.DataFrame(columns = ['transformer', 'number of overlaps'])

  for column in long_links.columns:
    num_overlaps = (short_links[column].values == long_links[column].values).sum() 
    new_data = {'transformer': column, 'number of overlaps': num_overlaps}
    overlaps = overlaps.append(new_data, ignore_index=True)
  
  return overlaps

In [33]:
overlaps = calculate_overlaps(links_long, links_short)

In [34]:
overlaps.sort_values(by=['number of overlaps'], ascending=False, inplace=True, ignore_index=True)

In [35]:
overlaps

,transformer,number of overlaps
0,sentence-transformers/nli-distilroberta-base-v2,23
1,sentence-transformers/LaBSE,21
2,sentence-transformers/stsb-roberta-base-v2,21
3,sentence-transformers/nli-roberta-large,21
4,sentence-transformers/nli-bert-large,20
...,...,...
76,sentence-transformers/nq-distilbert-base-v1,8
77,sentence-transformers/facebook-dpr-question_en...,8
78,sentence-transformers/msmarco-distilbert-base-...,7
79,sentence-transformers/facebook-dpr-ctx_encoder...,6


In [36]:
most_overlaps = overlaps.loc[0, 'transformer']

In [37]:
most_overlaps

'sentence-transformers/nli-distilroberta-base-v2'

## Results

In [38]:
def map_to_label(num_values):
  return np.asarray(['Weak' if item == 0 else 'Moderate' if item == 1 else 'Strong'  for item in num_values])

In [39]:
def create_mapping_table(label, values, finance_labels, ml_labels):
  new_dataset = pd.DataFrame(columns=[label] + finance_labels)

  new_dataset[label] = ml_labels
  length = len(finance_labels)

  for num in range(length):
    column = finance_labels[num]
    to_index = (num + 1) * len(ml_labels)
    from_index = to_index - len(ml_labels)
    new_dataset[column] = map_to_label(values[from_index:to_index])

  return new_dataset

In [40]:
# Define labels for the resulting tables
label = 'Mapping between finance and ML ethics'
finance_labels = ['Integrity', 'Objectivity', 'Competenece', 'Fairness', 'Confidentiality', 'Professionalism', 'Diligence']
ml_labels = ['Inclusive growth, sustainable development and weel-being', 'Human-centred values and fairness', 'Transparency and expalinability',
             'Robustness, security and safety', 'Accountability']

In [41]:
# Print results for mappings between ethical principles based on long definitions
values = links_long[most_overlaps].values
create_mapping_table(label, values, finance_labels, ml_labels)

,Mapping between finance and ML ethics,Integrity,Objectivity,Competenece,Fairness,Confidentiality,Professionalism,Diligence
0,"Inclusive growth, sustainable development and ...",Moderate,Moderate,Weak,Strong,Weak,Weak,Weak
1,Human-centred values and fairness,Strong,Strong,Moderate,Strong,Moderate,Strong,Moderate
2,Transparency and expalinability,Strong,Moderate,Strong,Strong,Moderate,Moderate,Moderate
3,"Robustness, security and safety",Strong,Weak,Weak,Moderate,Weak,Weak,Weak
4,Accountability,Strong,Weak,Weak,Strong,Weak,Moderate,Strong


In [42]:
# Print results for mappings between ethical principles based on short definitions
values = links_short[most_overlaps].values
create_mapping_table(label, values, finance_labels, ml_labels)

,Mapping between finance and ML ethics,Integrity,Objectivity,Competenece,Fairness,Confidentiality,Professionalism,Diligence
0,"Inclusive growth, sustainable development and ...",Moderate,Strong,Moderate,Strong,Weak,Weak,Weak
1,Human-centred values and fairness,Strong,Strong,Weak,Strong,Moderate,Moderate,Moderate
2,Transparency and expalinability,Strong,Strong,Weak,Strong,Strong,Moderate,Moderate
3,"Robustness, security and safety",Moderate,Strong,Weak,Moderate,Moderate,Weak,Weak
4,Accountability,Strong,Weak,Weak,Strong,Weak,Weak,Moderate


In [43]:
# Print results for mappings between ethical principles based on human-based annotations
values = [1,1,2,1,2, 2,2,0,0,0, 0,0,1,1,1, 2,2,1,1,2, 0,1,0,2,0, 0,0,1,2,2, 1,2,0,2,1]
create_mapping_table(label, values, finance_labels, ml_labels)

,Mapping between finance and ML ethics,Integrity,Objectivity,Competenece,Fairness,Confidentiality,Professionalism,Diligence
0,"Inclusive growth, sustainable development and ...",Moderate,Strong,Weak,Strong,Weak,Weak,Moderate
1,Human-centred values and fairness,Moderate,Strong,Weak,Strong,Moderate,Weak,Strong
2,Transparency and expalinability,Strong,Weak,Moderate,Moderate,Weak,Moderate,Weak
3,"Robustness, security and safety",Moderate,Weak,Moderate,Moderate,Strong,Strong,Strong
4,Accountability,Strong,Weak,Moderate,Strong,Weak,Strong,Moderate
